# October Code Jam

Prepared by Jeel Faldu, Jimmy Koester, and Raphael Lu

In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

#HuggingFace tools
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

from google.colab import output

import urllib.request

## Introduction

This project explores sentiment analysis using `distilbert-base-uncased-finetuned-sst-2-english` and `twitter-roberta-base-sentiment` from the Hugging Face Transformers library. The goal is to automatically classify text as positive, negative, or neutral using pretrained transformer models.

We analyzed sentiment in social media posts, compared multiple models, and visualized their performance and confidence levels. We also applied the models to a creative dataset, showcasing how NLP can reveal insights from real-world text such as tweets, song lyrics, or news articles.


## Device Loading

In [99]:
output.disable_custom_widget_manager()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device Loaded: {device}")

# model_1 = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# model_2 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

distil_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

Device Loaded: cuda


## Data Preprocessing

In data exploration, no duplicate entries were found. In the orignal data set, Tweets were encoded as
```
'nocode', 'happy', 'not-relevant', 'angry', 'disgust|angry',
'disgust', 'happy|surprise', 'sad', 'surprise', 'happy|sad',
'sad|disgust', 'sad|angry', 'sad|disgust|angry'
```

**Summary of Data Preprocessing:**
* Feilds were renamed `id`, `text`, and `label`
* `id` feild was ultimately dropped
* Target labels were re-coded into arrays using OHE:
     * e.g. with sentiments   
     `['angry' 'disgust' 'happy' 'nocode' 'not-relevant' 'sad' 'surprise']`,   
     a tensor of `[1 0 0 0 0 1 0]`
     indicates an `angry|sad` tweet

* Data were split using a 80:20 (train:test) ratio and stored in `X_train, X_test, y_train, y_test`

In [100]:
# Load dataset
data_url = 'https://raw.githubusercontent.com/jeelfaldu7/transformer_sentiment_analysis/refs/heads/main/dataset.csv'
df = pd.read_csv(data_url, header=None, names=['id', 'text', 'label'], sep=',')

# Display first few rows of the dataset
display(df.head())

,id,text,label
0,611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [101]:
# Display the summary of the dataset
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3085 entries, 0 to 3084
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3085 non-null   int64 
 1   text    3085 non-null   object
 2   label   3085 non-null   object
dtypes: int64(1), object(2)
memory usage: 72.4+ KB


In [102]:
df = df.drop('id', axis = 1)

In [103]:
# Display the shape of all the DataFrame
n_rows, n_cols = df.shape
print(f"The DataFrame has {n_rows} rows and {n_cols} columns")

The DataFrame has 3085 rows and 2 columns


In [104]:
# Check for duplicates in the 'final' DataFrame
df.duplicated().sum(),df['label'].unique()

(np.int64(37),
 array(['nocode', 'happy', 'not-relevant', 'angry', 'disgust|angry',
        'disgust', 'happy|surprise', 'sad', 'surprise', 'happy|sad',
        'sad|disgust', 'sad|angry', 'sad|disgust|angry'], dtype=object))

### Train-Test Split

In [105]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_df.drop('label', axis = 1)
X_test = test_df.drop('label', axis = 1)

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df['label'].str.split('|'))
y_test = mlb.transform(test_df['label'].str.split('|'))

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2468, 1) (2468, 7)
(617, 1) (617, 7)


In [106]:
print(mlb.classes_)
print(y_train[92])

['angry' 'disgust' 'happy' 'nocode' 'not-relevant' 'sad' 'surprise']
[0 0 1 0 0 0 0]


### Data Pre-processing for roBERTa model

* Tweets were processed using recommened method in model [documentation](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment).
     * Specifically usernames and hyperlinks were reduced to `@user` and `http` respectively
* Tweets were tokenized and stored in X2_train and X2_test

In [107]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [108]:
X2_train = X_train['text'].apply(preprocess)
X2_test = X_test['text'].apply(preprocess)

X2_train.head()

,text
2634,"I'm at @user in London, Greater London http"
2373,My favourite #oilpainting 'Tiger Tiger Burnin...
839,Currently @user to discuss #DefeatingDepressio...
2857,@user thanks for the Favourite!
761,Farron Gorey selfies before his performance at...


In [109]:
roberta_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [110]:
X2_train_tokens = roberta_tokenizer(X2_train.tolist(),
                                    padding=True,
                                    truncation=True,
                                    max_length=128,
                                    return_tensors="pt")

X2_test_tokens = roberta_tokenizer(X2_test.tolist(),
                                    padding=True,
                                    truncation=True,
                                    max_length=128,
                                    return_tensors="pt")

In [111]:
base_sentiments = base_sentiments = mlb.classes_

label2id = {label: idx for idx, label in enumerate(base_sentiments)}
id2label = {idx: label for idx, label in enumerate(base_sentiments)}

In [112]:
model_2 = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment",
    num_labels=len(base_sentiments),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
    problem_type="multi_label_classification"
).to(device)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([7, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([7]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Re-training the `roBERTa Model`

In [113]:
train_dataset = Dataset.from_dict({
    'input_ids': X2_train_tokens['input_ids'],
    'attention_mask': X2_train_tokens['attention_mask'],
    'labels': y_train.astype('float32').tolist()
})

test_dataset = Dataset.from_dict({
    'input_ids': X2_test_tokens['input_ids'],
    'attention_mask': X2_test_tokens['attention_mask'],
    'labels': y_test.astype('float32').tolist()
})

In [114]:
output.disable_custom_widget_manager()

training_args = TrainingArguments(output_dir="./sentiment-model",
                                  num_train_epochs=4,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  learning_rate=5e-5,
                                  weight_decay=0.1,
                                  eval_strategy="epoch",
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  report_to="none",
                                  seed = 42)

roberta_trainer = Trainer(model=model_2,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset,
                  processing_class=roberta_tokenizer)

roberta_trainer.train()

model_2.save_pretrained("./my-sentiment-model")
roberta_tokenizer.save_pretrained("./my-sentiment-model")



Epoch,Training Loss,Validation Loss
1,No log,0.169057
2,No log,0.158259
3,No log,0.180886
4,0.125500,0.178156


('./my-sentiment-model/tokenizer_config.json',
 './my-sentiment-model/special_tokens_map.json',
 './my-sentiment-model/vocab.json',
 './my-sentiment-model/merges.txt',
 './my-sentiment-model/added_tokens.json',
 './my-sentiment-model/tokenizer.json')

In [115]:
def predict_sentiments_debug(text, threshold=0.5):
    inputs = roberta_tokenizer(text, return_tensors="pt", truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model_2(**inputs)

    logits = outputs.logits
    probs = torch.sigmoid(logits)[0].cpu().numpy()

    print("All probabilities:")
    for idx, prob in enumerate(probs):
        sentiment = id2label[idx]
        print(f"  {sentiment}: {prob:.4f}")

    predictions = {}
    for idx, prob in enumerate(probs):
        sentiment = id2label[idx]
        if prob >= threshold:
            predictions[sentiment] = float(prob)

    return predictions

result = predict_sentiments_debug("I'm so scared of the recent politcs from both parties", threshold=0.3)
print(f"Sentiments: {result}")

All probabilities:
  angry: 0.4080
  disgust: 0.0515
  happy: 0.0828
  nocode: 0.0805
  not-relevant: 0.1071
  sad: 0.1999
  surprise: 0.1126
Sentiments: {'angry': 0.4079921841621399}
